In [1]:
import requests
from bs4 import BeautifulSoup

In [7]:
base_url="https://www.producthunt.com/leaderboard/monthly/2024/11"
headers={
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }

In [8]:
response=requests.get(base_url, headers=headers)

In [9]:
response.text

'<!DOCTYPE html><html lang="en" data-sentry-component="RootLayout" data-sentry-source-file="layout.tsx"><head><meta charSet="utf-8"/><meta name="viewport" content="width=device-width, initial-scale=1, minimum-scale=1"/><link rel="stylesheet" href="/_next/static/css/d559531d7c4cb85d.css" data-precedence="next"/><link rel="stylesheet" href="/_next/static/css/484c476526b01482.css" data-precedence="next"/><link rel="stylesheet" href="/_next/static/css/7d43cf00572574b7.css" data-precedence="next"/><link rel="stylesheet" href="/_next/static/css/c99d6c2024146960.css" data-precedence="next"/><link rel="stylesheet" href="/_next/static/css/4afdd19ce6c2ffda.css" data-precedence="next"/><link rel="stylesheet" href="/_next/static/css/2ebb67559d51e31c.css" data-precedence="next"/><link rel="stylesheet" href="/_next/static/css/fbff204ee5a8a652.css" data-precedence="next"/><link rel="stylesheet" href="/_next/static/css/886a1217ff9fd009.css" data-precedence="next"/><link rel="preload" as="script" fetch

In [10]:
soup=BeautifulSoup(response.content,'html.parser')

In [23]:
today_section = soup.find_all('section', attrs={'data-test': lambda x: x and x.startswith('post-item')})
result = []
for sec in today_section:
    result.append(sec.text)

result


['Postiz Your ultimate AI social media scheduling tool Open Source•Marketing automation•Social media marketing1121,644',
 'Vozo Video Translator Precise video translation, perfected with AI pilotSaaS•Artificial Intelligence•Video1511,378',
 'Commerce Pro by CapCut End-to-end content creation built for commerce successArtificial Intelligence•E-Commerce•Video801,158',
 'Superchat AI Agents for WhatsApp Business, Instagram & CoMessaging•Customer Communication•Marketing automation1781,687',
 "Lovable The world's first AI Full Stack EngineerDesign Tools•Developer Tools•Artificial Intelligence2871,616",
 'SWE-Kit Build your own Devin like software engineering agentOpen Source•Software Engineering•Artificial Intelligence38878',
 'Agree.com Free e-signature for everyoneFintech•SaaS•Artificial Intelligence1611,189',
 'Layer  Brain-Inspired plannerDesign Tools•Productivity•Task Management1491,043',
 'Raycast Notes Fast, light, and frictionless note-takingMac•Productivity•WritingGabe PerezI keep 

In [51]:
text

"Postiz Your ultimate AI social media scheduling tool Open Source•Marketing automation•Social media marketing1121,644\nVozo Video Translator Precise video translation, perfected with AI pilotSaaS•Artificial Intelligence•Video1511,378\nCommerce Pro by CapCut End-to-end content creation built for commerce successArtificial Intelligence•E-Commerce•Video801,158\nSuperchat AI Agents for WhatsApp Business, Instagram & CoMessaging•Customer Communication•Marketing automation1781,687\nLovable The world's first AI Full Stack EngineerDesign Tools•Developer Tools•Artificial Intelligence2871,616\nSWE-Kit Build your own Devin like software engineering agentOpen Source•Software Engineering•Artificial Intelligence38878\nAgree.com Free e-signature for everyoneFintech•SaaS•Artificial Intelligence1611,189\nLayer  Brain-Inspired plannerDesign Tools•Productivity•Task Management1491,043\nRaycast Notes Fast, light, and frictionless note-takingMac•Productivity•WritingGabe PerezI keep finding myself trying dif

In [57]:
import openai
from pydantic import BaseModel
from typing import List

In [60]:
from openai import OpenAI
client = OpenAI()

text = "\n".join(result)
prompt = f"""Extract and structure the following text into a list of mobile applications with the required attributes:
- Mobile App Name
- Description
- Tag 1
- Tag 2
- Tag 3

Each entry should be clearly separated and formatted correctly. Convert the given text into multiple structured records.

{text}"""

# Define a Pydantic model for structured response
class AppInfo(BaseModel):
    name: str
    description: str
    tag1: str
    tag2: str
    tag3: str

# Modify this to expect a list of responses
class AppList(BaseModel):
    apps: List[AppInfo]

    
completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant to crawl web."},
        {
            "role": "user",
            "content": prompt
        }
    ],
    response_format = AppList,
)

apps = completion.choices[0].message.parsed.apps
for app in apps:
    print(app)

name='Postiz' description='Your ultimate AI social media scheduling tool' tag1='Open Source' tag2='Marketing automation' tag3='Social media marketing'
name='Vozo' description='Precise video translation, perfected with AI pilot' tag1='SaaS' tag2='Artificial Intelligence' tag3='Video'
name='Commerce Pro by CapCut' description='End-to-end content creation built for commerce success' tag1='Artificial Intelligence' tag2='E-Commerce' tag3='Video'
name='Superchat' description='AI Agents for WhatsApp Business, Instagram & Co' tag1='Messaging' tag2='Customer Communication' tag3='Marketing automation'
name='Lovable' description="The world's first AI Full Stack Engineer" tag1='Design Tools' tag2='Developer Tools' tag3='Artificial Intelligence'
name='SWE-Kit' description='Build your own Devin like software engineering agent' tag1='Open Source' tag2='Software Engineering' tag3='Artificial Intelligence'
name='Agree.com' description='Free e-signature for everyone' tag1='Fintech' tag2='SaaS' tag3='Art

In [64]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 7.4 MB/s eta 0:00:00m eta 0:00:010:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.8/346.8 KB 7.2 MB/s eta 0:00:007.8 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 8.4 MB/s eta 0:00:00m eta 0:00:010:01:01


In [65]:
import pandas as pd

# Convert structured response to DataFrame
df = pd.DataFrame([app.dict() for app in apps])

# Display the DataFrame
print(df)

                      name                                        description  \
0                   Postiz      Your ultimate AI social media scheduling tool   
1                     Vozo  Precise video translation, perfected with AI p...   
2   Commerce Pro by CapCut  End-to-end content creation built for commerce...   
3                Superchat    AI Agents for WhatsApp Business, Instagram & Co   
4                  Lovable           The world's first AI Full Stack Engineer   
5                  SWE-Kit  Build your own Devin like software engineering...   
6                Agree.com                      Free e-signature for everyone   
7                    Layer                             Brain-Inspired planner   
8            Raycast Notes          Fast, light, and frictionless note-taking   
9                    Spiky  Real-time insights for faster, smarter sales d...   
10           BetterBugs.io     Capture bugs, record sessions, and fix with AI   
11                  Blanka  

/tmp/ipykernel_3542/3160641868.py:4: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  df = pd.DataFrame([app.dict() for app in apps])


In [32]:
print(completion.choices[0].message.content)

It looks like you have a list of various AI and tech products with their respective categories, features, and some user feedback or promotional messages. Here’s a brief summary of some of the items mentioned:

1. **Postiz**: An open-source social media scheduling tool aimed at marketing automation and social media marketing.

2. **Vozo**: A SaaS product specialized in precise video translation utilizing AI technology.

3. **Commerce Pro by CapCut**: A comprehensive content creation tool designed specifically for e-commerce success with a focus on video.

4. **Superchat**: AI agents for customer communications on platforms like WhatsApp and Instagram, enhancing marketing automation.

5. **Lovable**: Claimed to be the world's first AI Full Stack Engineer, offering design tools for developers.

6. **SWE-Kit**: An open-source agent that allows users to build their own Devin-like software engineering tool.

7. **Agree.com**: A platform providing free e-signature services for everyone, lever